In [1]:
import sys; sys.path.insert(0, '../')
import os
from scipy import io
from pde_diff import *
from robust_pde_diff import print_pde, RobustPCA, Robust_LRSTR

%pylab inline
pylab.rcParams['figure.figsize'] = (12, 8)
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

Populating the interactive namespace from numpy and matplotlib


In [2]:
DATA_PATH = '../experimental_data/NLS.mat'
data = io.loadmat(DATA_PATH)

t = data['tt'].flatten()[:,None]; dt = t[1]-t[0]; nt = len(t)
x = data['x'].flatten()[:,None]; dx = x[1]-x[0]; nx = len(x)

u = data['uu'].T
X, T = np.meshgrid(x,t)

dt = t[1]-t[0]
dx = x[2]-x[1]
n = len(x)
m = len(t)

In [3]:
# fig = figure()
# ax = fig.gca(projection='3d')
# surf = ax.plot_surface(X, T, abs(u), rstride=1, cstride=1, cmap=cm.coolwarm,
#     linewidth=0, antialiased=False)
# title(r'Nonlinear Schrodinger Equation: $|u(x,t)|$', fontsize = 20)
# xlabel('x', fontsize = 16)
# ylabel('t', fontsize = 16)

In [4]:
numpy.random.seed(0)
un = u + 0.01/np.sqrt(2)*std(real(u))*np.random.randn(u.shape[0],u.shape[1]) + 0.01/np.sqrt(2)*1j*std(imag(u))*np.random.randn(u.shape[0],u.shape[1])

In [5]:
un, E1 = RobustPCA(un, lam_2 = 0.3)

Please ensure that the shape of U is correct.
iteration:1, err:2289.6707600660666, nc_norm:545.1358975199537 eta1:0.006777382133811959
iteration:50, err:0.02005730182989053, nc_norm:608.5434528539637 eta1:0.7232751536179983
iteration:91, err:7.903915329710704e-06, nc_norm:308.99351298077977 eta1:36.008384526079695


In [6]:
width_x = 10
width_t = 10
deg = 4

m,n=u.shape

m2 = m-2*width_t
n2 = n-2*width_x

utn = np.zeros((m2,n2), dtype=np.complex64)
uxn = np.zeros((m2,n2), dtype=np.complex64)
uxxn = np.zeros((m2,n2), dtype=np.complex64)
uxxxn = np.zeros((m2,n2), dtype=np.complex64)

for i in range(n2):
    utn[:,i] = PolyDiff(real(un[:,i+width_x]), dt*np.arange(m), deg = deg, width = width_t)[:,0]
    utn[:,i] = utn[:,i]+1j*PolyDiff(imag(un[:,i+width_x]), dt*np.arange(m), deg = deg, width = width_t)[:,0]

for i in range(m2):
    x_derivatives = PolyDiff(real(un[i+width_t,:]), dx*np.arange(n), deg = deg, diff = 3, width = width_x)
    x_derivatives = x_derivatives+1j*PolyDiff(imag(un[i+width_t,:]), dx*np.arange(n), deg = deg, diff = 3, width = width_x)
    uxn[i,:] = x_derivatives[:,0]
    uxxn[i,:] = x_derivatives[:,1]
    uxxxn[i,:] = x_derivatives[:,2]

utn = np.reshape(utn, (n2*m2,1), order='F')
uxn = np.reshape(uxn, (n2*m2,1), order='F')
uxxn = np.reshape(uxxn, (n2*m2,1), order='F')
uxxxn = np.reshape(uxxxn, (n2*m2,1), order='F')
Xn_ders = np.hstack([np.ones((n2*m2,1)),uxn,uxxn,uxxxn])
Xn_data = np.hstack([np.reshape(un[width_t:m-width_t,width_x:n-width_x], (n2*m2,1), order='F'),
                     np.reshape(abs(un[width_t:m-width_t,width_x:n-width_x]), (n2*m2,1), order='F')])
derivatives_description = ['','u_{x}','u_{xx}', 'u_{xxx}']

Xn, rhs_des = build_Theta(Xn_data, Xn_ders, derivatives_description, 3, data_description = ['u','|u|'])

In [7]:
# ut = np.zeros((m,n), dtype=np.complex64)
# ux = np.zeros((m,n), dtype=np.complex64)
# uxx = np.zeros((m,n), dtype=np.complex64)
# uxxx = np.zeros((m,n), dtype=np.complex64)

# for i in range(n):
#     ut[:,i] = FiniteDiff(u[:,i], dt, 1)
# for i in range(m):
#     ux[i,:] = FiniteDiff(u[i,:], dx, 1)
#     uxx[i,:] = FiniteDiff(u[i,:], dx, 2)
#     uxxx[i,:] = FiniteDiff(u[i,:], dx, 3)
    
# ut = np.reshape(ut, (n*m,1), order='F')
# ux = np.reshape(ux, (n*m,1), order='F')
# uxx = np.reshape(uxx, (n*m,1), order='F')
# uxxx = np.reshape(uxxx, (n*m,1), order='F')
# X_ders = np.hstack([np.ones((n*m,1)),ux,uxx,uxxx])
# X_data = np.hstack([np.reshape(u, (n*m,1), order='F'), np.reshape(abs(u), (n*m,1), order='F')])
# derivatives_description = ['','u_{x}','u_{xx}', 'u_{xxx}']

# X, rhs_des = build_Theta(X_data, X_ders, derivatives_description, 3, data_description = ['u','|u|'])

In [9]:
# Solve with STRidge
lam = 1e-2
d_tol = 50

wn = TrainSTRidge(Xn,utn,lam,d_tol)
print("PDE derived using STRidge")
print_pde(wn, rhs_des)

PDE derived using STRidge
u_t = (0.000004 +0.527625i)u_{xx}
    + (-0.000003 +1.001721i)u|u|^2
   


In [10]:
err = np.abs(np.array([200*(0.527625-0.5), 100*(1.001721-1)]))
err.mean(), err.std()

(2.8485500000000052, 2.676449999999997)

In [7]:
# Not working ???
# Z, E1 = RobustPCA(u, lam_2 = 0.3)
# Ut, R, rhs_des = build_linear_system(Z, dt, dx, D=3, P=3, time_diff='FD', space_diff='FD')

# Solve noiseless data with RPCA + STR
# w = TrainSTRidge(R, ut, 10**-5, 500)
# print("PDE derived using STRidge")
# print_pde(w, rhs_des)

# Solve noiseless data with RPCA+LRSTR
utn = utn.reshape(m2, n2)
w, X, E2 = Robust_LRSTR(Xn, utn, rhs_des, lam_1 = 1e-2, lam_3 = 0.3, lam_4 = 1e-4, d_tol = 50)
print_pde(w, rhs_des)

iteration:1, err:8806.323438941707, nc_norm:2919.200555734746 eta2:0.001818358854185318
u_t = 
u_t = (-0.000133 +0.526551i)u_{xx}
    + (-0.000172 +0.999666i)u|u|^2
   
u_t = (-0.000126 +0.528545i)u_{xx}
    + (-0.000168 +1.003451i)u|u|^2
   
u_t = (-0.000119 +0.527655i)u_{xx}
    + (-0.000162 +1.001754i)u|u|^2
   
u_t = (-0.000114 +0.527654i)u_{xx}
    + (-0.000159 +1.001749i)u|u|^2
   
u_t = (-0.000111 +0.527657i)u_{xx}
    + (-0.000155 +1.001754i)u|u|^2
   
u_t = (-0.000107 +0.527659i)u_{xx}
    + (-0.000151 +1.001756i)u|u|^2
   
u_t = (-0.000105 +0.527659i)u_{xx}
    + (-0.000148 +1.001758i)u|u|^2
   
u_t = (-0.000103 +0.527660i)u_{xx}
    + (-0.000145 +1.001758i)u|u|^2
   
u_t = (-0.000101 +0.527660i)u_{xx}
    + (-0.000143 +1.001758i)u|u|^2
   
u_t = (-0.000100 +0.527659i)u_{xx}
    + (-0.000141 +1.001758i)u|u|^2
   
u_t = (-0.000099 +0.527659i)u_{xx}
    + (-0.000140 +1.001758i)u|u|^2
   
u_t = (-0.000098 +0.527659i)u_{xx}
    + (-0.000139 +1.001758i)u|u|^2
   
u_t = (-0.000098 

In [8]:
err = np.abs(np.array([200*(0.527660-0.5), 100*(1.001759-1)]))
err.mean(), err.std()

(2.853950000000005, 2.6780499999999985)

In [ ]:
# STR U+Noise: (2.8485500000000052, 2.676449999999997)
# DLrSR U+Noise: (2.853950000000005, 2.6780499999999985)